In [ ]:
#This is a random question generator: the format of questions is constant. it is in form of 'Does it relate to {random.choice(self.keywords[self.category]))

In [1]:
import tensorflow as tf
import numpy as np
import random
import json
from transformers import TFAutoModelForCausalLM, AutoTokenizer

2025-03-02 15:29:22.165320: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740929362.185423  195445 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740929362.191694  195445 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-02 15:29:22.213235: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/ladans/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jup

In [2]:
# Load JSON file
with open("keywords.json", "r") as f:
    data = json.load(f)

# Convert JSON list into a dictionary
keywords = {entry["category"]: [item["keyword"] for item in entry["words"]] for entry in data}

# Print to verify structure
print("Processed Keywords:", keywords)


Processed Keywords: {'country': ['afghanistan', 'albania', 'algeria', 'andorra', 'angola', 'antigua and barbuda', 'argentina', 'armenia', 'australia', 'austria', 'azerbaijan', 'bahrain', 'bangladesh', 'barbados', 'belarus', 'belgium', 'belize', 'benin', 'bhutan', 'bolivia', 'bosnia and herzegovina', 'botswana', 'brazil', 'brunei', 'bulgaria', 'burkina faso', 'burundi', 'cambodia', 'cameroon', 'canada', 'cape verde', 'central african republic', 'chad', 'chile', 'china', 'colombia', 'comoros', 'congo', 'costa rica', 'croatia', 'cuba', 'cyprus', 'czech republic', 'democratic republic of the congo', 'denmark', 'djibouti', 'dominica', 'dominican republic', 'ecuador', 'egypt', 'el salvador', 'england', 'equatorial guinea', 'eritrea', 'estonia', 'ethiopia', 'federated states of micronesia', 'finland', 'france', 'gabon', 'gambia', 'georgia', 'germany', 'ghana', 'greece', 'grenada', 'guatemala', 'guinea', 'guinea bissau', 'guyana', 'haiti', 'honduras', 'hungary', 'iceland', 'india', 'indonesia'

In [3]:
import json
import random

# Load JSON data
with open("keywords.json", "r") as f:
    data = json.load(f)

# Convert JSON to a structured dictionary
keywords = {}
for entry in data:
    category = entry["category"]
    words = [item["keyword"] for item in entry["words"]]
    keywords[category] = words

# Select a random category
category = random.choice(list(keywords.keys()))

# Select a secret word from that category
secret_word = random.choice(keywords[category])

print(f"Category: {category} (Secret Word: {secret_word})")

# Initialize game state
state = f"You are playing 20 Questions. Think of a {category}."
asked_questions = []

Category: landmark (Secret Word: colorado river)


In [14]:
import gym
import random
import numpy as np
from gym import spaces

class TwentyQuestionsEnv(gym.Env):
    def __init__(self, keywords):
        super(TwentyQuestionsEnv, self).__init__()

        # Store keywords dictionary
        self.keywords = keywords

        # Select a random category
        self.category = random.choice(list(self.keywords.keys()))
        self.secret_word = random.choice(self.keywords[self.category])

        # Define action & observation space
        self.action_space = spaces.Discrete(20)  # Max 20 questions
        self.observation_space = spaces.Box(low=0, high=1, shape=(20,), dtype=np.float32)

        # Initialize game state
        self.asked_questions = []
        self.done = False
        self.steps = 0
        self.max_steps = 20

    def step(self, action):
        """ Takes an action (asks a question using GPT-2). """
        if self.done:
            return np.array(self._get_state()), 0, True, {}

        # Generate a question using random choice
        context = ", ".join(self.keywords[self.category])  # Convert keywords to text input
        question = f"Does it relate to {random.choice(self.keywords[self.category])}?"

        # Simulate the answer
        answer = self._simulate_answer(question)

        # Calculate reward
        reward = self._calculate_reward(question, answer)

        # Track asked questions
        self.asked_questions.append((question, answer))

        # Check termination condition
        self.steps += 1
        if self.steps >= self.max_steps or answer == "Yes, correct!":
            self.done = True

        return np.array(self._get_state()), reward, self.done, {}

    def reset(self):
        """ Resets the environment for a new game. """
        self.category = random.choice(list(self.keywords.keys()))
        self.secret_word = random.choice(self.keywords[self.category])
        self.asked_questions = []
        self.done = False
        self.steps = 0
        return np.array(self._get_state())

    def _get_state(self):
        """ Returns the current game state representation. """
        return [len(self.asked_questions)]  # Simplified; can be extended


    def _simulate_answer(self, question):
        """ Simulates answers more realistically. """
        if any(keyword in question.lower() for keyword in self.keywords[self.category]):
                return random.choices(["Yes", "Maybe"], weights=[0.8, 0.2], k=1)[0]  # Positive feedback
        return random.choices(["No", "Maybe"], weights=[0.7, 0.3], k=1)[0]  # Some randomness


    def _calculate_reward(self, question, answer):
        if answer == "Yes":
            return 1  # Good question
        elif answer == "Maybe":
            return -0.5  # Unclear
        elif answer == "No":
            return -2  # Bad question
        return -1 # Default penalty for irrelevant questions



In [15]:
import tensorflow as tf
import numpy as np

class PPOAgent:
    def __init__(self, state_size, action_size, lr=0.001):
        self.state_size = state_size
        self.action_size = action_size
        self.lr = lr

        # Build policy and value networks
        self.policy_model = self._build_policy_model()
        self.value_model = self._build_value_model()

        # Initialize optimizers **only once**
        self.policy_optimizer = tf.keras.optimizers.Adam(learning_rate=self.lr)
        self.value_optimizer = tf.keras.optimizers.Adam(learning_rate=self.lr)

    def _build_policy_model(self):
        """Defines the policy network (actor)."""
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(64, activation='relu', input_shape=(self.state_size,)),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(self.action_size, activation='softmax')
        ])
        return model

    def _build_value_model(self):
        """Defines the value network (critic)."""
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(64, activation='relu', input_shape=(self.state_size,)),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(1, activation='linear')
        ])
        return model

    def get_action(self, state):
        """Selects an action based on the policy network."""
        state = np.expand_dims(state, axis=0)  # Ensure batch dimension
        probs = self.policy_model.predict(state, verbose=0)[0]  # Avoid TensorFlow warnings
        return np.random.choice(self.action_size, p=probs)

    def train(self, states, actions, rewards):
        """Updates the PPO policy using collected episode data."""
        states = np.array(states, dtype=np.float32)
        actions = np.array(actions, dtype=np.int32)
        rewards = np.array(rewards, dtype=np.float32)

        # Compute discounted rewards
        gamma = 0.99
        discounted_rewards = np.zeros_like(rewards, dtype=np.float32)
        cumulative = 0
        for t in reversed(range(len(rewards))):
            cumulative = rewards[t] + gamma * cumulative
            discounted_rewards[t] = cumulative

        # Normalize rewards
        discounted_rewards = (discounted_rewards - np.mean(discounted_rewards)) / (np.std(discounted_rewards) + 1e-8)

        # Compute advantages (A = R - V)
        values = self.value_model.predict(states, verbose=0).flatten()
        advantages = discounted_rewards - values

        # Convert to tensors
        states = tf.convert_to_tensor(states, dtype=tf.float32)
        actions = tf.convert_to_tensor(actions, dtype=tf.int32)
        advantages = tf.convert_to_tensor(advantages, dtype=tf.float32)
        discounted_rewards = tf.convert_to_tensor(discounted_rewards, dtype=tf.float32)

        # Perform training step using @tf.function to optimize execution
        self._train_step(states, actions, advantages, discounted_rewards)

    @tf.function
    def _train_step(self, states, actions, advantages, discounted_rewards):
        """Performs a single PPO training step using TensorFlow functions."""

        with tf.GradientTape() as policy_tape, tf.GradientTape() as value_tape:
            # Compute action probabilities
            probs = self.policy_model(states, training=True)
            action_masks = tf.one_hot(actions, self.action_size)
            selected_probs = tf.reduce_sum(probs * action_masks, axis=1)

            # Compute Policy Loss
            policy_loss = -tf.reduce_mean(tf.math.log(selected_probs + 1e-10) * advantages)

            # Compute Value Loss
            values = self.value_model(states, training=True)
            value_loss = tf.reduce_mean(tf.square(discounted_rewards - tf.squeeze(values)))

        # Compute gradients
        policy_grads = policy_tape.gradient(policy_loss, self.policy_model.trainable_variables)
        value_grads = value_tape.gradient(value_loss, self.value_model.trainable_variables)

        # Apply gradients
        self.policy_optimizer.apply_gradients(zip(policy_grads, self.policy_model.trainable_variables))
        self.value_optimizer.apply_gradients(zip(value_grads, self.value_model.trainable_variables))


In [16]:
# Initialize environment and agent
env = TwentyQuestionsEnv(keywords)
state_size = 1  # Simplified; can extend
action_size = 20
agent = PPOAgent(state_size, action_size)

episodes = 100  # Reduced for testing

for episode in range(episodes):
    state = env.reset()
    total_reward = 0
    rewards = []
    states = []
    actions = []

    for t in range(env.max_steps):
        action = agent.get_action(state)
        next_state, reward, done, _ = env.step(action)

        # Store experience
        states.append(state)
        actions.append(action)
        rewards.append(reward)

        state = next_state
        total_reward += reward

        if done:
            break

    # Update PPO policy after each episode
    agent.train(states, actions, rewards)

    print(f"Episode {episode+1}: Total Reward: {total_reward}")


Episode 1: Total Reward: 8.0
Episode 2: Total Reward: 9.5
Episode 3: Total Reward: 15.5
Episode 4: Total Reward: 15.5
Episode 5: Total Reward: 12.5
Episode 6: Total Reward: 12.5
Episode 7: Total Reward: 17.0
Episode 8: Total Reward: 12.5
Episode 9: Total Reward: 9.5
Episode 10: Total Reward: 15.5
Episode 11: Total Reward: 14.0
Episode 12: Total Reward: 12.5
Episode 13: Total Reward: 15.5
Episode 14: Total Reward: 14.0
Episode 15: Total Reward: 18.5
Episode 16: Total Reward: 11.0
Episode 17: Total Reward: 11.0
Episode 18: Total Reward: 20
Episode 19: Total Reward: 17.0
Episode 20: Total Reward: 11.0
Episode 21: Total Reward: 15.5
Episode 22: Total Reward: 14.0
Episode 23: Total Reward: 12.5
Episode 24: Total Reward: 12.5
Episode 25: Total Reward: 9.5
Episode 26: Total Reward: 14.0
Episode 27: Total Reward: 14.0
Episode 28: Total Reward: 12.5
Episode 29: Total Reward: 11.0
Episode 30: Total Reward: 14.0
Episode 31: Total Reward: 14.0
Episode 32: Total Reward: 14.0
Episode 33: Total Rewar